In [1]:
from __future__ import print_function
import os, sys
import json
import sqlite3
import traceback
import argparse

# Flag to disable value evaluation
DISABLE_VALUE = True
# Flag to disable distinct in select evaluation
DISABLE_DISTINCT = True
CLAUSE_KEYWORDS = ('select', 'from', 'where', 'group', 'order', 'limit', 'intersect', 'union', 'except')
JOIN_KEYWORDS = ('join', 'on', 'as')

WHERE_OPS = ('not', 'between', '=', '>', '<', '>=', '<=', '!=', 'in', 'like', 'is', 'exists')
UNIT_OPS = ('none', '-', '+', "*", '/')
AGG_OPS = ('none', 'max', 'min', 'count', 'sum', 'avg')
TABLE_TYPE = {
    'sql': "sql",
    'table_unit': "table_unit",
}

COND_OPS = ('and', 'or')
SQL_OPS = ('intersect', 'union', 'except')
ORDER_OPS = ('desc', 'asc')


HARDNESS = {
    "component1": ('where', 'group', 'order', 'limit', 'join', 'or', 'like'),
    "component2": ('except', 'union', 'intersect')
}


# sql {
#   'select': (isDistinct(bool), [(agg_id, val_unit), (agg_id, val_unit), ...])
#   'from': {'table_units': [table_unit1, table_unit2, ...], 'conds': condition}
#   'where': condition
#   'groupBy': [col_unit1, col_unit2, ...]
#   'orderBy': ('asc'/'desc', [val_unit1, val_unit2, ...])
#   'having': condition
#   'limit': None/limit value
#   'intersect': None/sql
#   'except': None/sql
#   'union': None/sql
# }

In [2]:
import json
import sqlite3
from nltk import word_tokenize

In [3]:
def tokenize(string):
    string = str(string)
    string = string.replace("\'", "\"")  # ensures all string values wrapped by "" problem??
    quote_idxs = [idx for idx, char in enumerate(string) if char == '"']
    assert len(quote_idxs) % 2 == 0, "Unexpected quote" # 싱글쿼터방지

    # keep string value as token
    vals = {}
    for i in range(len(quote_idxs)-1, -1, -2):
        qidx1 = quote_idxs[i-1]
        qidx2 = quote_idxs[i]
        val = string[qidx1: qidx2+1]
        key = "__val_{}_{}__".format(qidx1, qidx2)
        string = string[:qidx1] + key + string[qidx2+1:]
        vals[key] = val

    toks = [word.lower() for word in word_tokenize(string)]
    # replace with string value token
    for i in range(len(toks)):
        if toks[i] in vals:
            toks[i] = vals[toks[i]]

    # find if there exists !=, >=, <=
    eq_idxs = [idx for idx, tok in enumerate(toks) if tok == "="]
    eq_idxs.reverse()
    prefix = ('!', '>', '<')
    for eq_idx in eq_idxs:
        pre_tok = toks[eq_idx-1]
        if pre_tok in prefix:
            toks = toks[:eq_idx-1] + [pre_tok + "="] + toks[eq_idx+1: ]

    return toks

# toks = tokenize(query) <- *** 쿼리를 말그대로 조각내서 리스트형식으로 변환 

In [4]:
with open('C:/Users/yoonsub/Documents/nl2sql_data_labeling2.json' ,'r', encoding = 'utf-8') as f : 
    nl2sql = json.load(f)
    
    
nl2sql
# for 문으로 각각의 정보를 수집한후 다시 토큰화 하고 다시 붙인다.  # db_id 그대로 사용, querry 그대로 사용 # 쿼리토근 사용 

{'data': [{'db_id': 'seouldata_financial_0001',
   'utterance_id': 'q0001',
   'hardness': 'easy',
   'utterance_type': '04',
   'query': 'SELECT BUS_PSGR_CNT FROM TPSS_EMD_BUS',
   'utterance': '행정동별 버스 승객수를 보여줘',
   'values': [],
   'cols': [{'token': '버스 승객수', 'start': 5, 'column_index': 2}]},
  {'db_id': 'seouldata_financial_0002',
   'utterance_id': 'q0002',
   'hardness': 'easy',
   'utterance_type': '05',
   'query': 'SELECT CRNAME FROM CHILD_CARE_INFO WHERE NRTRROOMSIZE >= 350',
   'utterance': '보육실 면적이 350 이상인 어린이집은 어디야',
   'values': [{'token': '350', 'start': 8, 'column_index': 11}],
   'cols': [{'token': '어린이집', 'start': 16, 'column_index': 2}]},
  {'db_id': 'seouldata_financial_0003',
   'utterance_id': 'q0003',
   'hardness': 'easy',
   'utterance_type': '01',
   'query': 'SELECT ~* FROM BIG_JTY_UNIONPLATPLC WHERE WORK_DAY = 20111227000000',
   'utterance': '작업 일자가 2011년 12월 27일인 주택대장 조합대지위치 데이터만 보여줘',
   'values': [],
   'cols': []},
  {'db_id': 'seouldata_financial_0004

In [5]:
nl2sql['data'][0]['db_id']
nl2sql['data'][0]['utterance']

'행정동별 버스 승객수를 보여줘'

In [6]:
tokenize("보육실 면적이 350 이상인 어린이집은 어디야")

['보육실', '면적이', '350', '이상인', '어린이집은', '어디야']

In [7]:
# token 화 후 

query_toks = [] 
question_toks = [] 
db_id = [] 
query = [] 
question = [] 

for i in range(len(nl2sql['data'])) : 
        db_id.append(nl2sql['data'][i]['db_id'])
        query.append(nl2sql['data'][i]['query'])
        question.append(nl2sql['data'][i]['utterance'])   
        
        query_process = tokenize(query[i])
        query_toks.append(query_process)
        
        question_precess = tokenize(question[i])
        question_toks.append(question_precess)                  
   


In [8]:
# json 형태로 데이터 삽입하기 딕셔너리를 json.dump로 변환 
    
for i in range(0,3152) :         
    data['db_id'] = db_id[i]
    data['query'] = query[i]
    
    print(data)
    
    
    


NameError: name 'data' is not defined